In [1]:
import importlib
import pandas as pd
import numpy as np
import os
import sys
import pandas as pd
import plotly.io as pio
import seaborn as sns
import plotly.graph_objects as go

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats.distributions import chi2
from matplotlib import pyplot as plt
from scipy.stats import bernoulli, norm
from plotly.subplots import make_subplots
import plotly.express as px

%matplotlib inline

In [2]:

df1 = pd.read_csv('data_vintages1.csv')
df2 = pd.read_csv('data_vintages2.csv')

/Users/kirillgolubev/PycharmProjects/DeepLearningOnFingers/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# Объеденим два DataFrame
df = pd.concat([df1,df2])

В данных присутствует строка в которой почти все столбцы пропущены, поэтому от такой строчки лучше избавиться

In [4]:
df[df['user_id'].isna()]

,user_id,statement_num,gen_month_dt,gen_quarter_dt,clo_flg,def_flg,dlq_flg,act_flg,prev_clo_flg,prev_def_flg,prev_dlq_flg,prev_act_flg
1246634,NaN,1,0,0,0,1,0,0,0,NaN,NaN,NaN


In [5]:
df.drop([1246634],inplace = True)

In [6]:
df["statement_num"] = df["statement_num"].apply(lambda x: int(x))

Функции для предобработки данных

In [7]:
def f_0(x):
    """
    Функция нужна, чтобы избавиться от отрицательных значений 
    для переменных,
    где такое значение не предусмотрено
    """
    if x < 0:
        return 0
    return x


def preprocess(df):
    """
    Функция для преобразования признаков
    
    Parameters
    ----------
    df : DataFrame
        датафрейм с наблюдениями и целевой переменной


    Returns
    ---
    df: DataFrame 
        датафрейм с преобразованными признаками
    """

    # Заполнение пропусков
    df['avg_active_time'] = df['avg_active_time'].fillna(108.3456)
    df['min_term'] = df['min_term'].fillna(3.628)
    df['avg_term'] = df['avg_term'].fillna(91.501)

    for neg in ['avg_term', 'min_term', 'avg_active_time']:
        df[neg] = df[neg].apply(f_0)

    # Немного обрежем каждую переменную, из-за того, что некоторые из них имеют сильные выбросы
    df['avg_term'] = df['avg_term'].clip(
        lower=np.percentile(df['avg_term'], 0),
        upper=np.percentile(df['avg_term'], 99.5))
    
    df['min_term'] = df['min_term'].clip(
        lower=np.percentile(df['min_term'], 0),
        upper=np.percentile(df['min_term'], 95))
     
    features1 = features = [
    'max_util', 
    'avg_util',
    'credits_2y',
    'credits_4y',
    'bad_history_credits_flg',
    'other_util', 
    'avg_limit_mortgage',
    'min_limit_mfo', 
    'close_balance_amt',
    'soc_dem_score',
    'max_delq_cnt',
    'avg_active_time',
    'last_credit_time_years']
    
    for feat in features1:
        df[feat] = df[feat].clip(lower=np.percentile(df[feat], 1),
                                 upper=np.percentile(df[feat], 99))
    return df

Загрузим предсказанные pd, полученные при помощи модели из предыдущего дз 

In [8]:
df_scores = pd.read_csv('data_scores_and_features-0667850d-f03d-4d60-9ffc-93cb745050b4.csv')
df_scores.drop([32506,68471,93755],inplace = True) # Удалим три строчки, так как почти все признаки у них пустые
df_scores.reset_index(inplace = True)
df_scores.drop(['index'],axis=1,inplace=True)

df_scores = preprocess(df_scores)

# Загрузим предсказанные pd, полученные моделью из предыдущего дз
pd_pred = pd.read_csv('predictions_pd.csv')

In [9]:
pd_pred.drop(['Unnamed: 0'],axis=1,inplace=True)

In [10]:
sum(pd_pred.user_id == df_scores.user_id)

104817

In [11]:
# Добавим предсказания в исходный df
df_scores['pd_pred'] = pd_pred['pd']

In [12]:
df_scores['pd'] = df_scores['pd'].apply(lambda x: float('0.' + x[2:]))
df_scores['x_score'] = df_scores['x_score'].apply(lambda x: float('0.' + x[2:]))

In [13]:
df_scores[['user_id','pd','pd_pred','x_score']]

,user_id,pd,pd_pred,x_score
0,5269,0.039098,0.045165,0.127199
1,157747,0.002988,0.015628,0.624327
2,112119,0.063419,0.062266,0.166025
3,131663,0.053852,0.111411,0.097733
4,122778,0.046282,0.046438,0.458680
...,...,...,...,...
104812,39195,0.025842,0.055235,0.361921
104813,113647,0.085268,0.111078,0.196731
104814,158552,0.022367,0.054362,0.222133
104815,35628,0.051484,0.059158,0.247079


#### Бъем выборку на бакеты и джойним с данными с винтажами

In [14]:
df_scores = df_scores.sort_values(['pd_pred'], ignore_index=True)
df_scores['pd_bucket'] = np.minimum(df_scores.index // (df_scores.shape[0]//3),2)
print(df_scores.groupby(['pd_bucket'])['pd_pred'].mean())

df_scores = df_scores.sort_values(['x_score'], ignore_index=True)
df_scores['x_bucket'] = np.minimum(df_scores.index // (df_scores.shape[0]//3),2)
print(df_scores.groupby(['x_bucket'])['x_score'].mean())

df = df.merge(df_scores[['user_id', 'pd_bucket', 'x_bucket', 'pd_pred', 'x_score']], how='left', on='user_id').dropna()
df.head()

pd_bucket
0    0.024068
1    0.047762
2    0.099526
Name: pd_pred, dtype: float64
x_bucket
0    0.133271
1    0.220842
2    0.379455
Name: x_score, dtype: float64


,user_id,statement_num,gen_month_dt,gen_quarter_dt,clo_flg,def_flg,dlq_flg,act_flg,prev_clo_flg,prev_def_flg,prev_dlq_flg,prev_act_flg,pd_bucket,x_bucket,pd_pred,x_score
0,127336.0,19,2019-07-01,2019-07-01,0,0,0,1,0,0.0,0.0,1.0,1.0,2.0,0.053629,0.469410
1,35907.0,16,2019-04-01,2019-04-01,1,0,0,0,1,0.0,0.0,0.0,1.0,0.0,0.056802,0.157215
2,45228.0,21,2019-05-01,2019-04-01,0,0,0,1,0,0.0,0.0,1.0,1.0,2.0,0.046161,0.469583
3,39035.0,23,2019-07-01,2019-07-01,0,0,0,1,0,0.0,0.0,1.0,0.0,1.0,0.033263,0.210896
4,130780.0,19,2019-08-01,2019-07-01,1,0,0,0,1,0.0,0.0,0.0,1.0,1.0,0.049544,0.269492


Функция, создающая DataFrame нужного нам формата, с нужными кривыми

In [15]:
def lin_inter(x_1, y_1, x_2, y_2, x):
    return y_1 + (x - x_1) * (y_2 - y_1) / (x_2 - x_1)


def make_df(df, b1, b2):
    default_rate_data = df.eval('def_new_flg = def_flg * prev_act_flg', inplace=False)\
    .groupby(["gen_quarter_dt", 'x_bucket', "statement_num"])\
    .agg({"def_new_flg":np.sum, "prev_act_flg":np.sum, "user_id":np.count_nonzero})\
    .rename(columns = ({"def_new_flg":'def_cnt', "prev_act_flg":'prev_act_cnt', "user_id":"cnt"}))\
    .eval('default_rate = def_cnt / prev_act_cnt', inplace=False)

    # Исключаем "странные" поколения

    #exception_query = ''' ~((gen_quarter_dt == '2019-01-01') & (pd_bucket == 0)) '''
    #default_rate_data.query(exception_query, inplace=True)

    #exception_query = ''' ~((gen_quarter_dt == '2020-01-01') & (pd_bucket == 1)) '''
    #default_rate_data.query(exception_query, inplace=True)

    # Считаем среднее

    default_rate_mean = default_rate_data.groupby(['statement_num', 'x_bucket'])\
        .agg({'default_rate':np.mean})\
        .rename(columns = {'default_rate':'Среднее'})

    close_rate_data = df.eval('clo_new_flg = clo_flg * prev_act_flg', inplace=False)\
    .groupby(["gen_quarter_dt", 'x_bucket', "statement_num"])\
    .agg({"clo_new_flg":np.sum, "prev_act_flg":np.sum, "user_id":np.count_nonzero})\
    .rename(columns = ({"clo_new_flg":'clo_cnt', "prev_act_flg":'prev_act_cnt', "user_id":"cnt"}))\
    .eval('close_rate = clo_cnt / prev_act_cnt', inplace=False)

    #exception_query = ''' ~((gen_quarter_dt == '2019-01-01') & (pd_bucket == 0)) '''
    #close_rate_data.query(exception_query, inplace=True)

    close_rate_mean = close_rate_data.groupby(['statement_num', 'x_bucket'])\
    .agg({'close_rate':np.mean})\
    .rename(columns = {'close_rate':'Среднее'})

    df_ratio = df.groupby(["x_bucket","gen_month_dt", "statement_num"], as_index=False)\
        .agg({"dlq_flg":np.sum, "act_flg":np.sum, "user_id":np.count_nonzero})\
        .rename(columns = ({"dlq_flg":'dlq_cnt', "act_flg":'act_cnt', "user_id":"cnt"}))\
        .eval('dlq_ratio = dlq_cnt / act_cnt', inplace=False)


    dlq_mean = df_ratio.groupby(['statement_num', 'x_bucket'])\
    .agg({'dlq_ratio':np.mean})\
    .rename(columns = {'dlq_ratio':'Среднее'})

    hw_df = pd.DataFrame(
        columns=["statement_num", "def_rate", "dlnq_ratio", "clo_rate"])

    def_rate_arr = []
    for i in range(1, 25):
        def_rate_arr.append(
            lin_inter(0.133271, default_rate_mean.loc[i, b1], 0.220842,
                      default_rate_mean.loc[i, b2], 0.15)[0])

    dlnq_ratio_arr = []
    for i in range(1, 25):
        dlnq_ratio_arr.append(
            lin_inter(0.133271, dlq_mean.loc[i, b1], 0.220842,
                      dlq_mean.loc[i, b2], 0.15)[0])
    clo_rate_arr = []
    for i in range(1, 25):
        clo_rate_arr.append(
            lin_inter(0.133271, close_rate_mean.loc[i, b1], 0.220842,
                      close_rate_mean.loc[i, b2], 0.15)[0])

    hw_df["statement_num"] = list(range(1, 25))
    hw_df["def_rate"] = def_rate_arr
    hw_df["dlnq_ratio"] = dlnq_ratio_arr
    hw_df["clo_rate"] = clo_rate_arr

    return hw_df

Заданный pd = 5% лежит между первым и вторым бакетами

In [20]:
hm_df = make_df(df, 0.0, 1.0)
hm_df.to_excel("Table_with_x_score_m.xlsx")

In [19]:
hm_df

,statement_num,def_rate,dlnq_ratio,clo_rate
0,1,0.000000,0.091746,0.036304
1,2,0.000000,0.103928,0.023762
2,3,0.000000,0.118583,0.022103
3,4,0.029113,0.112139,0.022957
4,5,0.006503,0.114477,0.023195
5,6,0.007833,0.120490,0.025744
6,7,0.009812,0.128309,0.026347
7,8,0.008570,0.133262,0.026958
8,9,0.009936,0.139365,0.026223
9,10,0.011005,0.140083,0.028202
